# Coronagraph propagation

In this notebook, we do a propagation through a coronagaph, once by itself, and then through the entire optical setup of the THD2 testbed.

In [ ]:
import os
from matplotlib.colors import LogNorm
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
from Asterix import Asterix_root
from Asterix.utils import read_parameter_file
from Asterix.optics import butterworth_circle, Pupil, Coronagraph, DeformableMirror, mft, roundpupil, Testbed

In [ ]:
# Read configuration file
your_directory = Asterix_root
your_parameter_file_name = 'Example_param_file.ini'

parameter_file_path = os.path.join(your_directory, your_parameter_file_name)
config = read_parameter_file(parameter_file_path)#, NewCoronaconfig={'corona_type': 'wrapped_vortex'})

In [ ]:
Data_dir = config["Data_dir"]
onbench = config["onbench"]
modelconfig = config["modelconfig"]
DMconfig = config["DMconfig"]
Coronaconfig = config["Coronaconfig"]
Estimationconfig = config["Estimationconfig"]
Correctionconfig = config["Correctionconfig"]
Loopconfig = config["Loopconfig"]
SIMUconfig = config["SIMUconfig"]
Name_Experiment = SIMUconfig["Name_Experiment"]

In [ ]:
model_local_dir = os.path.join(Data_dir, "Model_local")
matrix_dir = os.path.join(Data_dir, "Interaction_Matrices")
result_dir = os.path.join(Data_dir, "Results", Name_Experiment)
labview_dir = os.path.join(Data_dir, "Labview")

In [ ]:
# Create all optical elements of the THD2 testbed
entrance_pupil = Pupil(modelconfig,
                       PupType=modelconfig['filename_instr_pup'],
                       angle_rotation=modelconfig['entrance_pup_rotation'],
                       Model_local_dir=model_local_dir)
DM1 = DeformableMirror(modelconfig, DMconfig, Name_DM='DM1', Model_local_dir=model_local_dir)
DM3 = DeformableMirror(modelconfig, DMconfig, Name_DM='DM3', Model_local_dir=model_local_dir)
corono = Coronagraph(modelconfig, Coronaconfig, Model_local_dir=model_local_dir)

# Concatenate into the full testbed optical system
thd2 = Testbed([entrance_pupil, DM1, DM3, corono],
               ["entrancepupil", "DM1", "DM3", "corono"])
print(f"\nCoronagraph type: {thd2.corono.corona_type}")

## Propagation through the coronagraph alone

In [ ]:
# PSF without FPM
direct_psf = corono.todetector_intensity(center_on_pixel=True, noFPM=True)
# Get normalization factor from direct PSF
max_psf = direct_psf.max()

In [ ]:
# Coronagraphic PSF (with FPM)
coro_psf = corono.todetector_intensity(center_on_pixel=True, noFPM=False)

In [ ]:
plt.figure(figsize=(16,8))

plt.subplot(1,2,1)
plt.imshow(direct_psf/max_psf, cmap='inferno', norm=LogNorm())
plt.title('Direct PSF')
plt.colorbar()

plt.subplot(1,2,2)
plt.imshow(coro_psf/max_psf, cmap='inferno', norm=LogNorm())
plt.title('Coronagraphic PSF')
plt.colorbar()

## Propagation through the full THD2 optical system

In [ ]:
# Calculate the PSF through the whole optical system
psf_after_testbed = thd2.todetector_intensity()

plt.imshow(psf_after_testbed, cmap='inferno')#, norm=LogNorm())
plt.title('Full testbed PSF (unnormalized)')
plt.colorbar()